## READ THE REQUIRED TWO TABLES 

In [2]:
import pandas as pd

In [3]:
data_train = pd.read_csv("C:/Users/mural/MDS/BLOCK6/DATA586/NEW/HDFS_1 (1)/HDFS_100k.log_structured.csv")
data_train.head(2)

,LineId,Date,Time,Pid,Level,Component,Content,EventId,EventTemplate
0,1,81109,12:00:00 AM,143,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,E5,Receiving block <*> src: /<*> dest: /<*>
1,2,81109,1:00:00 AM,35,INFO,dfs.FSNamesystem,BLOCK* NameSystem.allocateBlock: /mnt/hadoop/m...,E22,BLOCK* NameSystem.allocateBlock:<*>


In [4]:
data_label= pd.read_csv("C:/Users/mural/MDS/BLOCK6/DATA586/NEW/HDFS_1 (1)/anomaly_label.csv")
data_label.head(2)

,BlockId,Label
0,blk_-1608999687919862906,Normal
1,blk_7503483334202473044,Normal


#### WE SHOULD NOW EXTRACT THE BLOCKIDS FROM BOTH OF THEM AND MERGE THE DATAFRAME:

In [5]:
data_train['Content']

0         Receiving block blk_-1608999687919862906 src: ...
1         BLOCK* NameSystem.allocateBlock: /mnt/hadoop/m...
2         Receiving block blk_-1608999687919862906 src: ...
3         Receiving block blk_-1608999687919862906 src: ...
4         PacketResponder 1 for block blk_-1608999687919...
                                ...                        
104810    Verification succeeded for blk_-87387097785867...
104811    Verification succeeded for blk_642047611142564...
104812    Verification succeeded for blk_-23823897510323...
104813    Verification succeeded for blk_485603173001003...
104814    Verification succeeded for blk_-49047501208105...
Name: Content, Length: 104815, dtype: object

In [6]:
data_train['id']=data_train['Content'].str.extract(r'(?:blk_|blk_-)(\d+)')

In [7]:
# moving throug the data the regex we used are correct:

In [8]:
data_label['id']= data_label['BlockId'].str.extract(r'(?:blk_|blk_-)(\d+)')

#### Join both of these tables together

In [7]:
len(data_label)

575061

In [8]:
len(data_train)

104815

In [9]:
train = data_train[['id','EventId']]

In [10]:
# We should perform a left join from the values:
data = pd.merge(train, data_label, on='id', how='left')
data

,id,EventId,BlockId,Label
0,1608999687919862906,E5,blk_-1608999687919862906,Normal
1,1608999687919862906,E22,blk_-1608999687919862906,Normal
2,1608999687919862906,E5,blk_-1608999687919862906,Normal
3,1608999687919862906,E5,blk_-1608999687919862906,Normal
4,1608999687919862906,E11,blk_-1608999687919862906,Normal
...,...,...,...,...
104810,8738709778586756237,E2,blk_-8738709778586756237,Normal
104811,6420476111425645508,E2,blk_6420476111425645508,Normal
104812,2382389751032389929,E2,blk_-2382389751032389929,Normal
104813,4856031730010032819,E2,blk_4856031730010032819,Normal


### AGGREGRATE THE VALUES FOR THE EVENT IDS:

In [11]:
data_agg = data.groupby('BlockId')['EventId'].apply(lambda x: "[%s]" % ', '.join(x))
type(data_agg)

pandas.core.series.Series

In [12]:
data_agg = data_agg.to_frame()
data_agg = data_agg.reset_index()

In [13]:
data_agg

,BlockId,EventId
0,blk_-1001553972418305662,"[E22, E5, E5, E5, E26, E26, E11, E9, E11, E9, ..."
1,blk_-1010952805175971965,"[E22, E5, E5, E5, E26, E26, E26, E11, E9, E11,..."
2,blk_-1011482868748761910,"[E22, E5, E5, E5, E26, E26, E11, E9, E11, E9, ..."
3,blk_-1011537904811654030,"[E5, E22, E5, E5, E11, E9, E11, E9, E11, E9, E..."
4,blk_-1015291919896450721,"[E22, E5, E5, E5, E26, E9, E11, E9, E26, E26, ..."
...,...,...
7935,blk_989409441141247289,"[E22, E5, E5, E5, E26, E26, E26, E11, E9, E11,..."
7936,blk_989757635324841895,"[E5, E22, E5, E5, E11, E9, E11, E9, E11, E9, E..."
7937,blk_992037777600190687,"[E22, E5, E5, E5, E26, E26, E11, E9, E11, E9, ..."
7938,blk_993316727245644324,"[E22, E5, E5, E5, E11, E9, E11, E9, E11, E9, E..."


In [14]:
data_agg = pd.merge(data_agg, data[['BlockId','Label']], on= 'BlockId', how='inner').drop_duplicates()

#### FEATURE EXTRACTION WORK

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [16]:
vec = TfidfVectorizer()
x_train_features = vec.fit_transform(data_agg['EventId'])

In [19]:
display(x_train_features)

<7940x19 sparse matrix of type '<class 'numpy.float64'>'
	with 40978 stored elements in Compressed Sparse Row format>

In [17]:
y_train_features = data_agg['Label']

In [20]:
y_train.head(2)

7280      Normal
94382    Anomaly
Name: Label, dtype: object